This workbook requires no additional modules and should work with any _stable_ version of Julia.<br/>
The _ModInts_ module is from the Julia examples directory but is reproduced here in the _code_ subdirectory.<br/><br/>


# Why is Julia fast and can we retrofit to Python? 

## Type Inference

To generate fast code for a function f(x,y), the compiler needs to be able to infer the types of variables in f,
map them to hardware types (registers) where possible, and call specialized code paths for those types.

At compile-time, the compiler generally only knows types of x,y, but not the values, and it needs to be able
to cascade this information to infer types throughout f and in any functions called by f.

Julia and its standard library are designed so type inference is possible for code following straightforward rules.

Sometimes this requires subtle choices that would be painful to retrofit onto an existing language.



### _The return type of a function should ONLY depend on the types of its arguments_

### This needs to be run in Python

```python
# Import NUMPY for square roots

import numpy as np

# Look at type (in)stability

type(2 ** 3)
type(2 ** -3)

# How this is correct in Python but not in MATLAB

np.sqrt(1)
np.sqrt(-1)
np.sqrt(-1 + 0j)

```

### Let's see how Julia deals with this

In [14]:
typeof(2^3)

Int64

In [15]:
typeof(2^-3)

LoadError: DomainError
while loading In[15], in expression starting on line 1

In [16]:
sqrt(1)

1.0

In [17]:
sqrt(-1)

LoadError: DomainError
while loading In[17], in expression starting on line 1

In [18]:
sqrt(-1 + 0im)

0.0 + 1.0im

### and with the factorial

In [19]:
fac(n) = (n < 2) ? 1 : n*fac(n-1);
fac(20)

2432902008176640000

In [20]:
fac(21)

-4249290049419214848

### but

In [21]:
fac(big(21))

51090942171709440000

In [22]:
factorial(21)   # in the Base.combinatorics module

LoadError: OverflowError()
while loading In[22], in expression starting on line 1

- Not all mathematical operations on numbers in Julia produce the same _type_ of result.
- But Julia is consistant or an exception is raised.
- The outcome(s) are not without being the subject of a _great_ deal of debate.

In [23]:
2/4

0.5

In [24]:
4/2

2.0

In [25]:
4 ÷ 2            # x ÷ y    =>    div(x,y)  or  x div y

2

In [26]:
4//11 / 2//7     # divide 2 rationals

14//11

In [27]:
typeof(4//11 / 2//7)

Rational{Int64} (constructor with 1 method)

In [28]:
typeof(4//11 ÷ 2//7)

Int64

In [29]:
4//11 ÷ 2//7

1

In [30]:
typeof(4.0 ÷ 3.0)

Float64

---

### Delegation O-O paradigm, as contrasted with inheritence or association

We will look at how this works using one of the original Julia examples: Modular Integers

In [ ]:
# We need to be able to import the module by munging the LOAD PATH

In [31]:
cd("/Users/malcolm/notebooks.git/PYD/code");  # This is dependent on YOUR directory structure

In [32]:
push!(LOAD_PATH,".")

5-element Array{Union(ASCIIString,UTF8String),1}:
 "/Applications/Julia-0.3.11.app/Contents/Resources/julia/local/share/julia/site/v0.3"
 "/Applications/Julia-0.3.11.app/Contents/Resources/julia/share/julia/site/v0.3"      
 "."                                                                                  
 "."                                                                                  
 "."                                                                                  

In [33]:
import ModInts;    # "import" does not add functions to the MAIN class  
ms = ModInts       # so we need to assign an identifier, in a similar fash to Python's "as" statement

ModInts

In [34]:
; cat modints.jl

# This file is a part of Julia. License is MIT: http://julialang.org/license

module ModInts
export ModInt

immutable ModInt{n} <: Integer
    k::Int
    ModInt(k) = new(mod(k,n))
end

import Base.+, Base.-, Base.*

-{n}(a::ModInt{n}) = ModInt{n}(-a.k)
+{n}(a::ModInt{n}, b::ModInt{n}) = ModInt{n}(a.k+b.k)
-{n}(a::ModInt{n}, b::ModInt{n}) = ModInt{n}(a.k-b.k)
*{n}(a::ModInt{n}, b::ModInt{n}) = ModInt{n}(a.k*b.k)

Base.convert{n}(::Type{ModInt{n}}, i::Int) = ModInt{n}(i)
Base.promote_rule{n}(::Type{ModInt{n}}, ::Type{Int}) = ModInt{n}

Base.show{n}(io::IO, k::ModInt{n}) = print(io, "$(k.k) mod $n")
Base.showcompact(io::IO, k::ModInt) = print(io, k.k)

Base.inv{n}(a::ModInt{n}) = ModInt{n}(invmod(a.k, n))

end # module


In [10]:
m1 = ms.ModInt{13}(8);
m2 = ms.ModInt{13}(4);
m3 = ms.ModInt{13}(3);

In [35]:
m1 + m2*m3

7 mod 13

In [36]:
inv(m3)

9 mod 13

In [37]:
m1 + 1

9 mod 13

In [38]:
m1 + 1//2

LoadError: < not defined for ModInt{13}
while loading In[38], in expression starting on line 1

In [39]:
m1 + 2.0

LoadError: no promotion exists for ModInt{13} and Float64
while loading In[39], in expression starting on line 1

In [40]:
m1 + int(2.0)

10 mod 13

In [41]:
mm = [ms.ModInt{13}(rand(1:13)) for i = 1:100] 

100-element Array{Any,1}:
 12
  0
  3
 11
  7
  5
  1
  8
 12
  5
 12
  0
  0
  ⋮
  4
  8
 11
  4
  4
 12
  7
  9
  2
  7
  7
  2

In [42]:
ma = reshape(mm,10,10)

10x10 Array{Any,2}:
 12  12   8   5  12   3   0   3   9  11
  0   0   9  12  10   1   1   2   8   4
  3   0  12  11   1  12   2  12   6   4
 11   4   2   3   8   7   7   0   2  12
  7  12   0   7   2   7   3   2   6   7
  5  11   0   3   6   9   5   1  10   9
  1   9  11   7   3   9   3  12  11   2
  8   3   4   6   9   1   3   5   0   7
 12   5   4   7   5   4   7   6   4   7
  5   4   8  11   0  11  12   7   8   2

In [43]:
mx = [ms.ModInt{13}(rand(1:13)) for i = 1:10]

10-element Array{Any,1}:
 12
  7
  6
  3
 10
  0
  5
 11
  0
 10

In [44]:
mb = mx' .* ma .* mx 

10x10 Array{Any,2}:
 12   7   4  11  10  0   0   6  0   7
  0   0   1   5  11  0   9  11  0   7
  8   0   3   3   8  0   8  12  0   6
  6   6  10   1   6  0   1   0  0   9
  8   8   0   2   5  0   7  12  0  11
  0   0   0   0   0  0   0   0  0   0
  8   3   5   1   7  0  10  10  0   9
  3  10   4   3   2  0   9   7  0   3
  0   0   0   0   0  0   0   0  0   0
  2   7  12   5   0  0   2   3  0   5

In [45]:
mc = map(x -> x^3, mb)

10x10 Array{ModInt{13},2}:
 12   5  12  5  12  0   0   8  0  5
  0   0   1  8   5  0   1   5  0  5
  5   0   1  1   5  0   5  12  0  8
  8   8  12  1   8  0   1   0  0  1
  5   5   0  8   8  0   5  12  0  5
  0   0   0  0   0  0   0   0  0  0
  5   1   8  1   5  0  12  12  0  1
  1  12  12  1   8  0   1   5  0  1
  0   0   0  0   0  0   0   0  0  0
  8   5  12  8   0  0   8   1  0  8

In [46]:
mc[1:5,1:5]

5x5 Array{ModInt{13},2}:
 12  5  12  5  12
  0  0   1  8   5
  5  0   1  1   5
  8  8  12  1   8
  5  5   0  8   8

## Staged code generation

### Parsed code ------> AST ------> AST (typed) ------> LLVM IR ------> [Native code]

```julia
# 1 == 1 parsed as ==(1,1)

# Try some of the following:

@code_lowered 1 == 1       # or  ==(1,1)
@code_typed   1 == 1
@code_llvm    1 == 1  
@code_native  1 == 1

```

In [47]:
@code_native 1 == 1

	.section	__TEXT,__text,regular,pure_instructions
Filename: promotion.jl
Source line: 198
	push	RBP
	mov	RBP, RSP
	cmp	RDI, RSI
Source line: 198
	sete	AL
	pop	RBP
	ret


In [48]:
@code_native 1 == 1.0

	.section	__TEXT,__text,regular,pure_instructions
Filename: float.jl
Source line: 170
	push	RBP
	mov	RBP, RSP
Source line: 170
	cvtsi2sd	XMM1, RDI
	ucomisd	XMM1, XMM0
	setnp	AL
	sete	CL
	and	CL, AL
	cvttsd2si	RAX, XMM1
	cmp	RAX, RDI
	sete	AL
	and	AL, CL
	pop	RBP
	ret


In [ ]:
# By the way

1 == 1.0

In [ ]:
1 === 1.0

In [1]:
# Insert a simple insert function

incr(x) = x + 1

incr (generic function with 1 method)

In [3]:
@code_native incr(1)   # Code for an integer argument

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[1]
Source line: 3
	push	RBP
	mov	RBP, RSP
Source line: 3
	lea	RAX, QWORD PTR [RDI + 1]
	pop	RBP
	ret


In [4]:
@code_native incr(1.0)  # Code for an float

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[1]
Source line: 3
	push	RBP
	mov	RBP, RSP
	movabs	RAX, 4357800352
Source line: 3
	addsd	XMM0, QWORD PTR [RAX]
	pop	RBP
	ret


In [11]:
incr(m1)    # This works too

9 mod 13

In [13]:
@code_native incr(m1)   # Here is the 'native' code

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[1]
Source line: 3
	push	RBP
	mov	RBP, RSP
Source line: 3
	inc	RDI
	movabs	RSI, 5675921253449092805
	mov	RAX, RDI
	imul	RSI
	mov	RAX, RDX
	shr	RAX, 63
	sar	RDX, 2
	add	RDX, RAX
	imul	RAX, RDX, 13
	mov	RCX, RDI
	sub	RCX, RAX
	lea	RAX, QWORD PTR [RCX + 13]
	imul	RSI
	mov	RAX, RDX
	shr	RAX, 63
	sar	RDX, 2
	add	RDX, RAX
	imul	RAX, RDX, 13
	neg	RAX
	test	RDI, RDI
	lea	RAX, QWORD PTR [RCX + RAX + 13]
	cmovns	RAX, RCX
	pop	RBP
	ret


In [49]:
# We help the compiler if we restrict (type) the arguments
#
# Define a fibonanicci function (recursively)

function fib(n :: Integer)
  @assert n > 0
  return (n < 3) ? 1 : fib(n-1) + fib(n-2)
end

fib(20)

6765

In [50]:
fib(-20)

LoadError: assertion failed: n > 0
while loading In[50], in expression starting on line 1

In [ ]:
fib(2.3)

In [51]:
macroexpand(:(@assert n > 0))

:(if n > 0
        nothing
    else 
        Base.error("assertion failed: n > 0")
    end)

In [52]:
@code_native(fib(20))

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[49]
Source line: 6
	push	RBP
	mov	RBP, RSP
	push	R15
	push	R14
	push	RBX
	push	RAX
	mov	RBX, RDI
	test	RBX, RBX
Source line: 6
	jle	64
Source line: 7
	cmp	RBX, 2
	jle	38
	lea	RDI, QWORD PTR [RBX - 1]
	movabs	R15, 4369299760
	call	R15
	mov	R14, RAX
	add	RBX, -2
	mov	RDI, RBX
	call	R15
	add	RAX, R14
	jmpq	5
	mov	EAX, 1
	add	RSP, 8
	pop	RBX
	pop	R14
	pop	R15
	pop	RBP
	ret
Source line: 6
	movabs	RAX, 4327730368
	mov	EDI, 16
	call	RAX
	movabs	RCX, 140683490906752
	mov	QWORD PTR [RAX], RCX
	movabs	RCX, 140683625480160
	mov	QWORD PTR [RAX + 8], RCX
	movabs	RCX, 4327674208
	mov	RDI, RAX
	mov	ESI, 6
	call	RCX


In [53]:
macroexpand(:(@printf "The value of fib(%d) is %d" 20 fib(20)))

quote 
    #221#out = Base.Printf.STDOUT
    #222###x#25200 = 20
    #223###x#25201 = fib(20)
    local #228#neg, #227#pt, #226#len, #220#exp, #224#do_out, #225#args
    Base.Printf.write(#221#out,"The value of fib(")
    if Base.Printf.isfinite(#222###x#25200)
        (#224#do_out,#225#args) = Base.Printf.decode_dec(#221#out,#222###x#25200,"",0,-1,'d')
        if #224#do_out
            (#226#len,#227#pt,#228#neg) = #225#args
            #228#neg && Base.Printf.write(#221#out,'-')
            Base.Printf.write(#221#out,Base.Printf.pointer(Base.Printf.DIGITS),#227#pt)
        end
    else 
        Base.Printf.write(#221#out,begin  # printf.jl, line 143:
                if Base.Printf.isnan(#222###x#25200)
                    "NaN"
                else 
                    if (#222###x#25200 Base.Printf.< 0)
                        "-Inf"
                    else 
                        "Inf"
                    end
                end
            end)
    end
    Base.Printf.write(#22

---

### Library calls have zero-overhead

In [54]:
systime() = ccall((:time,("libc")), Int32, ())

systime (generic function with 1 method)

In [55]:
systime()

1462550533

In [56]:
@code_native systime()

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[54]
Source line: 1
	push	RBP
	mov	RBP, RSP
	movabs	RAX, 140735708865772
Source line: 1
	call	RAX
	pop	RBP
	ret
